In [1]:
from pycocotools.coco import COCO
import glob, os, sys, zipfile
import urllib.request
import shutil
import numpy as np
import skimage.io as io
import json
import cv2
os.chdir("./processed")
json_file='../annotations/instances_train2017.json'
data=json.load(open(json_file,'r'))
data_2 = data["annotations"]

In [13]:
import math
bg = cv2.imread('../mask.jpg')
for file in glob.glob("../all_images/*.jpg"):
    # read file
    img = cv2.imread(file)
    ID = int(file.strip("/all_images/.jpg"))
    
    # read as mask
    for item in data_2:
        if item["image_id"] == ID:
            bigarr = np.array(item["segmentation"])
            bigarr = np.array(bigarr[0])
            pts = bigarr.reshape(-1,1,2).astype(np.int32)
            mask = np.zeros(img.shape[:2], np.uint8)
            cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1)
            cv2.imwrite("mask_1.png", mask)
            #cv2.polylines(img, [pts], True, (0, 255, 255))
            break
    
    # possible resizes
    rows, cols, channels = bg.shape
    r1, c1, ch1 = img.shape
    rat_r = 0
    rat_c = 0
    if r1 > rows:
        rat_r = float(r1 / rows)
    if c1 > cols:
        rat_c = float(c1 / cols)
    ratio = max(rat_r, rat_c)
    
    # actual resize
    masked = cv2.imread('mask_1.png')
    new = cv2.imread('../mask.jpg')
    if ratio != 0:
        img = cv2.resize(img, (math.floor(r1/ratio), math.floor(c1/ratio)))
        masked = cv2.resize(masked, (math.floor(r1/ratio), math.floor(c1/ratio)))
    
    mrows, mcols, mchs = masked.shape
    # read pixel
    for i in range(rows):
        for j in range(cols):
            if i >= mrows or j >= mcols:
                new[i,j,m] = bg[i,j,m]
            elif masked[i,j].all() == 0:
                for m in range(channels):
                    new[i,j,m] = bg[i,j,m]
            else:
                for m in range(channels):
                    new[i,j,m] = img[i,j,m]
                    
    # write image
    cv2.imwrite("{}_final.png".format(ID),new)